----
<img src="../../files/lseg.svg" class="lseg-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery Layer
There are 3 layers in the Refinitiv Data Library: [Delivery](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/3-Delivery), [Content](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/2-Content) and [Access](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/1-Access). This notebook demonstrates how to use the Definition object of the Delivery layer in conjunction with with the functions `get_data` and `get_history` and the Instrument Pricing Analytics (IPA) product.

The Delivery layer provides a simplified data request interface for REST APIs, designed for FinCoders.

#### IPA

IPA (otherwise known as the Quantitative Analytics (QA) API Family) is a powerful product provided by LSEG through its Data Platform REST APIs and [Data Libraries](https://developers.lseg.com/en/api-catalog?x1=w_products&q1=devportal%3Aproducts%2Frefinitiv-data-libraries) that provides calculators for financial operations including [FX Cross Contracts](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts). Thanks to this service, we can recreate any calculation found in, for example, the FWDS app.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

 You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

----

## Some Imports to start with

In [1]:
import refinitiv.data as rd  # This is LSEG's Data and Analytics' API wrapper, called the Refinitiv Data Library for Python. You can update this library with the comand `!pip install refinitiv-data --upgrade`

import pandas as pd  # We need `pandas` for datafame and array manipilations.
import datetime # We use these to manipulate time values

%matplotlib inline
import plotly.graph_objects as go # `plotly` is a library used to render interactive graphs
import IPython
import ipywidgets as widgets
from refinitiv_widgets import Select, MultiSelect, Button, TextFieldAutosuggest
import copy

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
# rd.open_session(
#     name="platform.rdpRDP",  # name="desktop.workspace4", "platform.rdph"
#     config_name="C:/Example.DataLibrary.Python-main/Configuration/refinitiv-data.config.json")
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f53612bacd0 {name='codebook'}>

### Endpoints
You can reach the IPA API Family via the Quantitative Analytics set of endpoints (they all start with "https://api.refinitiv.com/data/quantitative-analytics").

#### [IPA Curves Service: Construction of the FX Forward Curve (Forward Curves Endpoint)](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-fx-forward-curve)

The "Construction of the FX Forward Curve" set of functionalities is linked with the "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves" Endpoint. It allows us to collect data with 'absolute' (e.g.: "2023-11-09") or 'relative' (e.g.: ) `startTenor` dates; let's have a look at an example with an 'absolute' `startTenor` date.

In [3]:
frwd_crv_deliv_req = rd.delivery.endpoint_request.Definition(
    method=rd.delivery.endpoint_request.RequestMethod.POST,
    url="https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/forward-curves",
    body_parameters={
      "universe": [
        {
          "curveDefinition": {
          "baseCurrency": "EUR",
          "quotedCurrency": "GBP"
          },
        "forwardDefinitions": [
          {
            "startTenor": "2023-11-09",  # "0D"
          }
        ]
        }
      ]
    }
  ).get_data()
if frwd_crv_deliv_req.is_success:
    frwd_crv_deliv_df = pd.json_normalize(
        frwd_crv_deliv_req.data.raw['data'][0]['forwardCurves'][0]['curve']['curvePoints'])
    frwd_crv_deliv_df.columns.name=frwd_crv_deliv_req.data.raw['data'][0]['curveDefinition']['crossCurrencyDefinitions'][0]['name']
else:
    display(frwd_crv_deliv_req.errors)

You can find information about the curve returned with `curveDefinition` and `curveParameters`:

In [4]:
print("curveDefinition")
display(frwd_crv_deliv_req.data.raw['data'][0]['curveDefinition'])
print("curveParameters")
display(frwd_crv_deliv_req.data.raw['data'][0]['curveParameters'])

curveDefinition


{'baseCurrency': 'EUR',
 'quotedCurrency': 'GBP',
 'crossCurrencyDefinitions': [{'baseCurrency': 'EUR',
   'baseIndexName': 'EURIBOR',
   'name': 'EUR GBP FxForward',
   'quotedCurrency': 'GBP',
   'quotedIndexName': 'LIBOR',
   'source': 'Refinitiv',
   'isNonDeliverable': False,
   'mainConstituentAssetClass': 'FxForward',
   'riskType': 'CrossCurrency',
   'id': 'c2aecf88-3b26-405e-a7f0-30fa2c2918b7',
   'ignoreExistingDefinition': False}]}

curveParameters


{'valuationDate': '2023-11-15',
 'interpolationMode': 'Linear',
 'extrapolationMode': 'Constant',
 'turnAdjustments': {},
 'ignorePivotCurrencyHolidays': False,
 'useDelayedDataIfDenied': False,
 'ignoreInvalidInstrument': True,
 'marketDataLookBack': {'value': 96, 'unit': 'Hour'}}

In [5]:
frwd_crv_deliv_df

EUR GBP FxForward,tenor,startDate,endDate,instruments,swapPoint.bid,swapPoint.ask,swapPoint.mid,outright.bid,outright.ask,outright.mid
0,ON,2023-11-09,2023-11-10,[{'instrumentCode': 'EURGBPON='}],0.000000,0.000000,0.000000,0.872240,0.872746,0.872493
1,SPOT,2023-11-13,2023-11-13,NaN,0.000000,0.000000,0.000000,0.872240,0.872746,0.872493
2,SN,2023-11-13,2023-11-14,[{'instrumentCode': 'EURGBPSN='}],0.000000,0.000000,0.000000,0.872240,0.872746,0.872493
3,SW,2023-11-13,2023-11-20,[{'instrumentCode': 'EURGBPSW='}],1.400000,1.480000,1.440000,0.872386,0.872888,0.872637
4,2W,2023-11-13,2023-11-27,[{'instrumentCode': 'EURGBP2W='}],3.414286,3.615714,3.515000,0.872587,0.873102,0.872845
5,3W,2023-11-13,2023-12-04,[{'instrumentCode': 'EURGBP3W='}],5.468571,5.785714,5.627143,0.872793,0.873319,0.873056
6,1M,2023-11-13,2023-12-13,[{'instrumentCode': 'EURGBP1M='}],8.120000,8.415000,8.267500,0.873058,0.873582,0.873320
7,2M,2023-11-13,2024-01-16,[{'instrumentCode': 'EURGBP2M='}],20.515000,20.850667,20.682833,0.874298,0.874825,0.874561
8,3M,2023-11-13,2024-02-13,[{'instrumentCode': 'EURGBP3M='}],29.472727,29.902727,29.687727,0.875193,0.875730,0.875462
9,4M,2023-11-13,2024-03-13,[{'instrumentCode': 'EURGBP4M='}],38.230714,38.845000,38.537857,0.876069,0.876625,0.876347


#### FX Forward Curves through time
Let's now look into an example collecting several curves in one go, using 'relative' `startTenor` dates.

In [6]:
def get_fwd_surf_data(
    intrvl,
    rnge,
    debug=True,
    dta_of_intrst="swapPoint.bid",
    base_curr="EUR",
    quoted_curr="GBP"):
    """
    intrvl (str): 'D' for days, 'W' for weeks, 'M' for months, 'Y' for years.
    rnge (str): We're looking until `rnge` `intrvl` forward, be it 10 Y, 4 D, 2 M, ...
    dta_of_intrst (str): Data of interest, to select from 'swapPoint.bid', 'swapPoint.ask', 'swapPoint.mid', 'outright.bid', 'outright.ask', or 'outright.mid'.
    debug (bool): If True, displays original `curves_df` dataframe.
    """

    frwd_srf_deliv_req = rd.delivery.endpoint_request.Definition(
        method=rd.delivery.endpoint_request.RequestMethod.POST,
        url="https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/forward-curves",
        body_parameters={"universe": [{
          "curveDefinition": {
              "baseCurrency": base_curr,
              "quotedCurrency": quoted_curr,
          },
          "forwardDefinitions": [{"startTenor": f"{i}{intrvl}"} for i in range(0,rnge)]}]
          }).get_data()

    if debug is True:
        print(f"base_curr: {base_curr}, quoted_curr: {quoted_curr}, dta_of_intrst: {dta_of_intrst}, rnge: {rnge}, intrvl: {intrvl}")
        display(frwd_srf_deliv_req.data.raw)
    
    if frwd_srf_deliv_req.is_success:
        raw_data = frwd_srf_deliv_req.data.raw['data'][0]
        frwd_crv_deliv_df2_lst = [pd.json_normalize(
            raw_data['forwardCurves'][i]['curve']['curvePoints'])
            for i in range(0,rnge)]
        for i in range(0,rnge):
            len_raw_data = len(raw_data['forwardCurves'][i]['curve']['curvePoints'])
            frwd_crv_deliv_df2_lst[i]["curve"]=[
                raw_data['curveDefinition']['crossCurrencyDefinitions'][0]['name'] + f" {i}{intrvl}"
                for j in range(len_raw_data)]
            frwd_crv_deliv_df2_lst[i][f"startTenorIn{intrvl}"] = [i for j in range(len_raw_data)]
            if i == 0:
                curves_df = frwd_crv_deliv_df2_lst[i]
            else:
                curves_df = curves_df.append(frwd_crv_deliv_df2_lst[i])

        if debug is True:
            display(curves_df)
        
        surface_df = pd.DataFrame(
            index=list(
                curves_df[curves_df[f"startTenorIn{intrvl}"]==curves_df[f"startTenorIn{intrvl}"][0].iloc[0]]["tenor"]))
        for i in curves_df.groupby([f"startTenorIn{intrvl}"]):
            surface_df[i[0]] = list(i[1][dta_of_intrst].values)
        surface_df.columns.name=f"startTenorIn{intrvl}"
        surface_df.index.name="tenor"
        
        titl = frwd_srf_deliv_req.data.raw['data'][0]['curveDefinition']['crossCurrencyDefinitions'][0]['name']

    else:
        display(frwd_srf_deliv_req.errors)

    return {'frwd_crv_deliv_req': frwd_srf_deliv_req,
            'curves_df': curves_df,
            'surface_df': surface_df,
            'titl': titl,
            'dta_of_intrst': dta_of_intrst}

In [7]:
def plot_surface(surfaces,  # surface_df
                 surfaceTag,  # titl
                 tenors,
                 dta_of_intrst="swapPoint.bid",
                 tenors_in_order=['ON', 'SPOT', 'SN', 'SW', '2W', '3W', '1M', '2M', '3M', '4M', '5M', '6M', '7M', '8M', '9M', '10M', '11M', '1Y', '15M', '18M', '21M', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']):
    
    _surfaces = surfaces.loc[[i for i in tenors_in_order if i in tenors and i in surfaces.index]]

    fig = go.Figure(data=[go.Surface(z=_surfaces.values)])

    fig.update_layout(
        template="plotly_dark",
        title=surfaceTag,
        autosize=False,
        width=1000, height=600,
        margin=dict(l=5, r=5, b=5, t=80),
        scene=dict(
            xaxis_title=f"x: {_surfaces.columns.name}",
            yaxis_title=f"y: {_surfaces.index.name}",
            zaxis_title=f"z: {dta_of_intrst}",
            yaxis=dict(ticktext=list(_surfaces.index),
                       tickvals=[i for i in range(len(_surfaces))])))

    fig.show()

In [13]:
def FX_fwrd_srfce(
    debug=False,
    base_currency="EUR",
    quote_currency="GBP",
    tenors_in_order=['ON', 'SPOT', 'SN', 'SW', '2W', '3W', '1M', '2M', '3M', '4M', '5M', '6M', '7M', '8M', '9M', '10M', '11M', '1Y', '15M', '18M', '21M', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']):

    fx_pair_out = widgets.Output()
    fx_pair_choice = TextFieldAutosuggest(placeholder='Currency FX Pair', filters=['FX'])
    display(fx_pair_choice, fx_pair_out) # This displays the box
    
    # print('Data of interest (Swap Points or Outrights; Ask, Bid, or Mid?)')
    dta_of_intrst_select_out = widgets.Output()
    dta_of_intrst_choice = Select(placeholder='Data of interest (Swap Points or Outrights; Ask, Bid, or Mid?)', width=300.0)
    dta_of_intrst_choice.data = [
        {'value': i, 'label': i, 'items': []}
         for i in ['swapPoint.bid', 'swapPoint.ask', 'swapPoint.mid', 'outright.bid', 'outright.ask', 'outright.mid']]
    display(dta_of_intrst_choice, dta_of_intrst_select_out)

    tenor_select_out = widgets.Output()
    intrvl_choice = Select(placeholder='Start Tenor Interval', width=300.0)
    intrvl_choice.data = [{'value': i, 'label': i, 'items': []}
                          for i in ['D', 'W', 'M', 'Y']]
    display(intrvl_choice, tenor_select_out)

    range_select_out = widgets.Output()
    rng_choice = Select(placeholder='Range', width=300.0)
    # slct_dta = [{'label': f"{1}", 'value': 1, 'selected': True}]
    # for i in range(2, 11):
    #     slct_dta.append({'label': f"{i}", 'value': i})
    rng_choice.data = [{'value': str(i), 'label': str(i), 'items': []}
                       for i in range(2, 31)]
    display(rng_choice, range_select_out)

    # print('Tenor(s)')
    tenor_choice_data = copy.deepcopy([
        {'value': i, 'label': i, 'items': []}
        for i in tenors_in_order])
    tenor_chose = MultiSelect(placeholder='Tenor(s)', data=tenor_choice_data, no_relation=True)
    display(tenor_chose)

    # create widgets
    button = Button('Create/Update Surface')
    button_output = widgets.Output()

    # create click handler
    def click_handler(a):
        with button_output:

            IPython.display.clear_output(wait=True)
            
            bs_cur = fx_pair_choice.value.split("=",1)[0][:3]
            qu_cur = fx_pair_choice.value.split("=",1)[0][3:6]
            if len(qu_cur) == 0:
                qu_cur = "USD"
            
            try:
                BASE_CCY_check = rd.get_data(fx_pair_choice.value, "BASE_CCY")
            except:
                print(f"{fx_pair_choice.value} is not an FX pair. Fetching data foir it gave the following error:")
                rd.get_data(fx_pair_choice.value, "BASE_CCY")
            if len(BASE_CCY_check) <= 0:
                print(f"Note that {fx_pair_choice.value}' is not a 'pure' FX pair(such as 'EURGBP=' or 'JPY='). Fetching data for it may give an error.")
                rd.get_data(fx_pair_choice.value, "BASE_CCY")

            surfs_data = get_fwd_surf_data(
                base_curr=bs_cur,
                quoted_curr=qu_cur,
                debug=debug,
                intrvl=intrvl_choice.value,
                rnge=int(rng_choice.value),
                dta_of_intrst=dta_of_intrst_choice.value)

            plot_surface(
                surfaces=surfs_data['surface_df'],
                surfaceTag=surfs_data['titl'],
                tenors=tenor_chose.value,
                dta_of_intrst=surfs_data['dta_of_intrst'])
            print(f"Updated on {datetime.datetime.now().strftime('%Y-%m-%d %H:%M.%s')}")

    # refister click handler for button
    button.on_click(click_handler)
    display(button)

    # display our widgets
    display(button_output)

In [14]:
FX_fwrd_srfce(debug=False)

TextFieldAutosuggest(value='', filters=['FX'], placeholder='Currency FX Pair', profile='', tooltip='')

Output()

Select(data=[{'value': 'swapPoint.bid', 'label': 'swapPoint.bid', 'items': []}, {'value': 'swapPoint.ask', 'la…

Output()

Select(data=[{'value': 'D', 'label': 'D', 'items': []}, {'value': 'W', 'label': 'W', 'items': []}, {'value': '…

...

Select(data=[{'value': '2', 'label': '2', 'items': []}, {'value': '3', 'label': '3', 'items': []}, {'value': '…

Output()

MultiSelect(data=[{'value': 'ON', 'label': 'ON', 'items': []}, {'value': 'SPOT', 'label': 'SPOT', 'items': []}…

Button(height=0.0, tooltip='', value='Create/Update Surface', width=0.0)

Output()

# Conclusion

## Reference of Endpoints' Documentation as of 2023-11-07:

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-fx-forward-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/zc-curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-zero-coupon-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-forward-zero-coupon-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/forward-curves:

This is in progress.

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/credit-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-commodity-forward-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/inflation-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-inflation-curve